In [ ]:
#MASKD Best Submission : 
#Train CascadeRCNN Resnest101 :

#Train data : Iterations : 2200 BaseLR : 0.0005 - Save Model
#Load weights of saved model and 
#Train data : Iterations : 400 BaseLR : 0.00025

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [ ]:
os.chdir("/content/drive/My Drive/blitz/maskd")

In [ ]:
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/train_images.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/val_images.zip
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/test_images.zip

In [ ]:
!unzip -q train_images.zip
!unzip -q val_images.zip
!unzip -q test_images.zip

In [ ]:
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/train.json
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/val.json
!wget -q https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/maskd/v0.1/test.json

In [ ]:
!nvidia-smi

In [ ]:

# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
pwd

In [ ]:
!git clone https://github.com/zhanghang1989/detectron2-ResNeSt.git

In [ ]:
cd detectron2-ResNeSt

In [ ]:
!python -m pip install -e . 

In [ ]:
# install detectron2:
!pip install detectron2==0.1.2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

In [ ]:

# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
os.chdir("/content/drive/My Drive/blitz/maskd")

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/drive/My Drive/blitz/maskd/train.json", "/content/drive/My Drive/blitz/maskd/train_images")
register_coco_instances("my_dataset_val", {}, "/content/drive/My Drive/blitz/maskd/val.json", "/content/drive/My Drive/blitz/maskd/val_images")

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file("/content/drive/My Drive/blitz/maskd/detectron2-ResNeSt/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_101_FPN_syncBN_1x.yaml")
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))


cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "https://s3.us-west-1.wasabisys.com/resnest/detectron/resnest101_detectron-486f69a8.pth" # Let training initialize from model zoo
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
# cfg.MODEL.WEIGHTS = "/content/drive/My Drive/blitz/maskd/output/cascadeRCNN_/train_2200(0.0005)_400(0.00025)_val_200(0.00025).pth" # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 400  # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # only has one class (ballon)


In [ ]:
pwd

In [ ]:
cfg.OUTPUT_DIR = "./output/cascadeRCNN_/"
# cfg.OUTPUT_DIR = "./output/faster_cascade_rcnn_ResNeSt_200_FPN_syncbn_range-scale_1x"

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)


In [ ]:
trainer.train()

In [ ]:
cfg.OUTPUT_DIR

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set the testing threshold for this model
cfg.DATASETS.TEST = ("my_dataset_val", )
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
eval_folder = "./output/cascadeRCNN_/evaluations"
os.makedirs(eval_folder, exist_ok=True)
evaluator = COCOEvaluator("my_dataset_val", cfg, False, output_dir="./output/cascadeRCNN_/evaluations/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
valResults = inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
import json
with open('/content/drive/My Drive/blitz/maskd/train.json') as json_file:
    train_anno = json.load(json_file)

In [ ]:
train_anno['categories']

In [ ]:
import os
import json
import torch, torchvision
import detectron2
import numpy as np
import cv2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.structures import Boxes, BoxMode 
import pycocotools.mask as mask_util
from pycocotools.coco import COCO
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import Boxes, BoxMode, pairwise_iou
import pycocotools.mask as mask_util


In [ ]:
json_file = "/content/drive/My Drive/blitz/maskd/train.json"
coco_api = COCO(json_file)
dataset_name = "my_dataset_train"
id_map = None
if dataset_name is not None:
    meta = MetadataCatalog.get(dataset_name)
    cat_ids = sorted(coco_api.getCatIds())
    cats = coco_api.loadCats(cat_ids)
    # The categories in a custom json file may not be sorted.
    thing_classes = [c["name"] for c in sorted(cats, key=lambda x: x["id"])]
    meta.thing_classes = thing_classes

    # In COCO, certain category ids are artificially removed,
    # and by convention they are always ignored.
    # We deal with COCO's id issue and translate
    # the category ids to contiguous ids in [0, 80).

    # It works by looking at the "categories" field in the json, therefore
    # if users' own json also have incontiguous ids, we'll
    # apply this mapping as well but print a warning.
    if not (min(cat_ids) == 1 and max(cat_ids) == len(cat_ids)):
        if "coco" not in dataset_name:
          print("We will apply mapping to you")
    id_map = {v: i for i, v in enumerate(cat_ids)}
    meta.thing_dataset_id_to_contiguous_id = id_map

In [ ]:
dataset_name = "my_dataset_train"
meta = MetadataCatalog.get(dataset_name)
if hasattr(meta, "thing_dataset_id_to_contiguous_id"):
            print("Hello")
            reverse_id_mapping = {
                v: k for k, v in meta.thing_dataset_id_to_contiguous_id.items()
            }

In [ ]:
meta

In [ ]:
reverse_id_mapping

In [ ]:
model_path = '/content/drive/My Drive/blitz/maskd/output/cascadeRCNN_/model_final.pth'
test_images_path = "/content/drive/My Drive/blitz/maskd/test_images/"
cpu_device = torch.device("cpu")

In [ ]:
#Solving extra category error in retinaNet
def instances_to_coco_json(instances, img_id):
    num_instance = len(instances)
    if num_instance == 0:
        return []

    boxes = instances.pred_boxes.tensor.numpy()
    boxes = BoxMode.convert(boxes, BoxMode.XYXY_ABS, BoxMode.XYWH_ABS)
    boxes = boxes.tolist()
    scores = instances.scores.tolist()
    classes = instances.pred_classes.tolist()

    has_mask = instances.has("pred_masks")
    if has_mask:
        rles = [
            mask_util.encode(np.array(mask[:, :, None], order="F", dtype="uint8"))[0]
            for mask in instances.pred_masks
        ]
        for rle in rles:
            rle["counts"] = rle["counts"].decode("utf-8")

    has_keypoints = instances.has("pred_keypoints")
    if has_keypoints:
        keypoints = instances.pred_keypoints

    results = []
    for k in range(num_instance):
        result = {
            "image_id": img_id,
            "category_id": classes[k],
            "bbox": boxes[k],
            "score": scores[k],
        }
        if has_mask:
            result["segmentation"] = rles[k]
    
        results.append(result)
    return results







cfg = get_cfg()
cfg.merge_from_file("/content/drive/My Drive/blitz/maskd/detectron2-ResNeSt/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_101_FPN_syncBN_1x.yaml")
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0   # set the testing threshold for this model
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
# cfg.MODEL.DEVICE = "cpu"
predictor = DefaultPredictor(cfg)

print("Done")

results = []

for i in tqdm(os.listdir(test_images_path)):
    # print(" i ",i)
    img_path =test_images_path + i
    im = cv2.imread(img_path)
    # print(img_path)
    if(im is not None):
        outputs = predictor(im)
        instances = outputs["instances"].to(cpu_device)
        # print(outputs)
        fname = i.split('.')[0]
        result = instances_to_coco_json(instances,fname)
        if(len(result)!=0):
            for p in result:
                matchId = p['category_id']
                p['category_id'] = reverse_id_mapping[matchId]
                results.append(p) 
    
    

In [ ]:
len(results)

In [ ]:
with open('/content/drive/My Drive/blitz/maskd/test.json') as json_file:
    x = json.load(json_file)

for i in x['images']:
    i['fname'] = i['file_name'].split('.')[0]

for i in results:
    image_mapper = i['image_id']
    for j in x['images']:
        if(image_mapper == j['fname']):
            i['image_id'] = j['id']

with open("/content/drive/My Drive/blitz/maskd/submissions/train_5000_0thres.json", "w") as write_file:
    json.dump(results, write_file)
